In [1]:
import urllib3
import certifi

import pandas as pd
pd.set_option('display.max_columns', 500)

import psycopg2 as pg
import config as creds
import sys

import sqlalchemy
from sqlalchemy import create_engine

In [2]:
http = urllib3.PoolManager(
    cert_reqs='CERT_REQUIRED',
    ca_certs=certifi.where()
    )

DOWNLOAD LATEST BUILDINGS DATASET FROM DOEE WEBSITE:

In [3]:
df = pd.read_csv('https://opendata.arcgis.com/datasets/aba010cff7fe4d4cb369a54b56cd7544_45.csv')
df.to_csv('C:/Users/rv14996/Power-Rangers/data/Building_Energy_Benchmarks.csv')

In [4]:
df.shape

(15732, 71)

In [5]:
df.head()

,X,Y,OBJECTID,PID,DCREALPROPERTYID,PMPROPERTYID,PROPERTYNAME,PMPARENTPROPERTYID,PARENTPROPERTYNAME,REPORTINGYEAR,REPORTSTATUS,ADDRESSOFRECORD,OWNEROFRECORD,WARD,REPORTEDADDRESS,CITY,STATE,POSTALCODE,YEARBUILT,PRIMARYPROPERTYTYPE_SELFSELECT,PRIMARYPROPERTYTYPE_EPACALC,TAXRECORDFLOORAREA,REPORTEDBUILDINGGROSSFLOORAREA,ENERGYSTARSCORE,SITEEUI_KBTU_FT,WEATHERNORMALZEDSITEEUI_KBTUFT,SOURCEEUI_KBTU_FT,WEATHERNORMALZEDSOUREUI_KBTUFT,TOTGHGEMISSIONS_METRICTONSCO2E,TOTGHGEMISSINTENSITY_KGCO2EFT,WATERSCORE_MFPROPERTIES,WATERUSE_ALLWATERSOURCES_KGAL,NATURALGASUSE_THERMS,FUELOILANDDIESELFUELUSEKBTU,METEREDAREAS_ENERGY,METEREDAREAS_WATER,LATITUDE,LONGITUDE,ADDRESSID,XCOORD,YCOORD,GIS_LAST_MOD_DTTM,DISTRCHILLEDWATER_KBTU,DISTRHOTWATER_KBTU,DISTRSTEAM_KBTU,ELECTRICITYUSE_RENEWABLE_KWH,ELECTRICITYUSE_GRID_KWH,NATURALGAS_KBTU_JANUARY,NATURALGAS_KBTU_FEBRUARY,NATURALGAS_KBTU_MARCH,NATURALGAS_KBTU_APRIL,NATURALGAS_KBTU_MAY,NATURALGAS_KBTU_JUNE,NATURALGAS_KBTU_JULY,NATURALGAS_KBTU_AUGUST,NATURALGAS_KBTU_SEPTEMBER,NATURALGAS_KBTU_OCTOBER,NATURALGAS_KBTU_NOVEMBER,NATURALGAS_KBTU_DECEMBER,ELECTRICITYUSE_KBTU_JANUARY,ELECTRICITYUSE_KBTU_FEBRUARY,ELECTRICITYUSE_KBTU_MARCH,ELECTRICITYUSE_KBTU_APRIL,ELECTRICITYUSE_KBTU_MAY,ELECTRICITYUSE_KBTU_JUNE,ELECTRICITYUSE_KBTU_JULY,ELECTRICITYUSE_KBTU_AUGUST,ELECTRICITYUSE_KBTU_SEPTEMBER,ELECTRICITYUSE_KBTU_OCTOBER,ELECTRICITYUSE_KBTU_NOVEMBER,ELECTRICITYUSE_KBTU_DECEMBER
0,-77.062954,38.903090,1,PM3514246,1188_0118,3514246.0,Waterfront,NaN,NaN,2017,In Compliance,1010 - 1014 WISCONSIN AV NW,WATERFRONT CENTER LP,2.0,1010-1014 Wisconsin Ave NW,WASHINGTON,DC,20007,1985.0,Office,Office,320940.0,207863.0,62.0,77.8,82.4,241.4,255.8,1777.5,8.6,NaN,6749.5,2853.629361,0.0,Whole Building,Whole Building,38.903083,-77.062952,278784.0,394539.532342,137249.388973,2020/11/01 09:02:33+00,0.0,0.0,0.0,0.0,4656224.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-76.989560,38.855388,2,DC58730921,5873_0921,NaN,NaN,NaN,NaN,2017,No Report Received,2500 POMEROY RD SE,ELVANS MANOR ASSOCIATES,8.0,NaN,WASHINGTON,DC,20020,1968.0,NaN,NaN,102272.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.855380,-76.989557,289831.0,400906.407085,131952.203196,2020/11/01 09:02:33+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-76.984751,38.854253,3,DC58740051,5874_0051,NaN,NaN,NaN,NaN,2017,No Report Received,2400 ELVANS RD SE,STANTON OAKS ASSOCIATES,8.0,NaN,WASHINGTON,DC,20020,1966.0,NaN,NaN,156698.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.854245,-76.984748,NaN,401323.839975,131826.238309,2020/11/01 09:02:33+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-77.041928,38.902246,4,PM1419693,0106_0048,1419693.0,1800 K Street,NaN,NaN,2017,In Compliance,1800 K ST NW,RAR2 1800 K ST NW DC INC,2.0,"1800 K Street, NW",WASHINGTON,DC,20006,1971.0,Office,Office,242124.0,235914.0,NaN,32.7,32.7,102.8,102.8,856.4,3.6,NaN,3743.4,0.000000,0.0,Whole Building,Whole Building,38.902238,-77.041926,300930.0,396363.331176,137154.624517,2020/11/01 09:02:33+00,0.0,0.0,0.0,0.0,2262703.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-77.024196,38.880608,5,PM3411637,0390_0053,3411637.0,NBL Associates,NaN,NaN,2017,In Compliance,800 9TH ST SW,NBL ASSOCIATES LP,6.0,"800 9th Street, SW",WASHINGTON,DC,20024,1986.0,Office,Office,160748.0,120838.0,82.0,68.1,68.6,213.9,215.3,913.3,7.6,NaN,2219.2,0.000000,0.0,Whole Building,Whole Building,38.880600,-77.024194,276806.0,397900.772041,134752.014064,2020/11/01 09:02:33+00,0.0,0.0,0.0,0.0,2412838.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ENDPOINT="dc-energy-benchmarking-project.cetnqm9k1imt.us-east-1.rds.amazonaws.com"
PORT="5432"
USR="postgres"
REGION="us-east-1"
DBNAME="dc_energy_data"

In [ ]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
    
    conn = pg.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor 

In [ ]:
 conn, cursor = connect()

In [ ]:
# cursor.execute(""" CREATE TABLE IF NOT EXISTS raw_energy_data(
# X text,
# Y text,
# OBJECTID integer, 
# PID text, 
# DCREALPROPERTYID text,
# PMPROPERTYID decimal,
# PROPERTYNAME text,
# PMPARENTPROPERTYID text,
# PARENTPROPERTYNAME text,
# REPORTINGYEAR integer,
# REPORTSTATUS text,
# ADDRESSOFRECORD text,
# OWNEROFRECORD text,
# WARD numeric,
# REPORTEDADDRESS text,
# CITY text,
# STATE text,
# POSTALCODE text,
# YEARBUILT integer,
# PRIMARYPROPERTYTYPE_SELFSELECT text,
# PRIMARYPROPERTYTYPE_EPACALC text,
# TAXRECORDFLOORAREA integer,
# REPORTEDBUILDINGGROSSFLOORAREA double precision,
# ENERGYSTARSCORE double precision,
# SITEEUI_KBTU_FT double precision,
# WEATHERNORMALZEDSITEEUI_KBTUFT double precision,
# SOURCEEUI_KBTU_FT double precision,
# WEATHERNORMALZEDSOUREUI_KBTUFT double precision, 
# TOTGHGEMISSIONS_METRICTONSCO2E double precision,
# TOTGHGEMISSINTENSITY_KGCO2EFT double precision,
# WATERSCORE_MFPROPERTIES double precision,
# WATERUSE_ALLWATERSOURCES_KGAL double precision,
# NATURALGASUSE_THERMS double precision,
# FUELOILANDDIESELFUELUSEKBTU double precision,
# METEREDAREAS_ENERGY text,
# METEREDAREAS_WATER text,
# LATITUDE text,
# LONGITUDE text,
# ADDRESSID integer,
# XCOORD double precision,
# YCOORD double precision,
# GIS_LAST_MOD_DTTM text,
# DISTRCHILLEDWATER_KBTU double precision,
# DISTRHOTWATER_KBTU double precision,
# DISTRSTEAM_KBTU double precision,
# ELECTRICITYUSE_RENEWABLE_KWH double precision,
# ELECTRICITYUSE_GRID_KWH double precision,
# NATURALGAS_KBTU_JANUARY double precision,
# NATURALGAS_KBTU_FEBRUARY double precision,
# NATURALGAS_KBTU_MARCH double precision,
# NATURALGAS_KBTU_APRIL double precision,
# NATURALGAS_KBTU_MAY double precision,
# NATURALGAS_KBTU_JUNE double precision,
# NATURALGAS_KBTU_JULY double precision,
# NATURALGAS_KBTU_AUGUST double precision,
# NATURALGAS_KBTU_SEPTEMBER double precision,
# NATURALGAS_KBTU_OCTOBER double precision,
# NATURALGAS_KBTU_NOVEMBER double precision,
# NATURALGAS_KBTU_DECEMBER double precision,
# ELECTRICITYUSE_KBTU_JANUARY double precision,
# ELECTRICITYUSE_KBTU_FEBRUARY double precision,
# ELECTRICITYUSE_KBTU_MARCH double precision,
# ELECTRICITYUSE_KBTU_APRIL double precision,
# ELECTRICITYUSE_KBTU_MAY double precision,
# ELECTRICITYUSE_KBTU_JUNE double precision,
# ELECTRICITYUSE_KBTU_JULY double precision,
# ELECTRICITYUSE_KBTU_AUGUST double precision,
# ELECTRICITYUSE_KBTU_SEPTEMBER double precision,
# ELECTRICITYUSE_KBTU_OCTOBER double precision,
# ELECTRICITYUSE_KBTU_NOVEMBER double precision,
# ELECTRICITYUSE_KBTU_DECEMBER double precision);""")

# conn.commit()

In [ ]:
f_contents = open('C:/Users/rv14996/Power-Rangers/data/Building_Energy_Benchmarks.csv', 'r')
cursor.copy_from(f_contents, 'raw_energy_data', sep=',')
conn.commit()

In [ ]:
cursor.execute("""SELECT * FROM raw_energy_data LIMIT 5""")

In [ ]:
query_results = cursor.fetchall()
print(query_results)

In [ ]:
conn.close()